In [ ]:
%pip install mistralai gradio pydantic chromadb

In [ ]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

api_key = ""


In [ ]:
model = "mistral-medium"

client = MistralClient(api_key=api_key)

messages = [
    ChatMessage(role="user", content="What is the best French cheese?")
]
# for chunk in client.chat_stream(model=model, messages=messages):
# No streaming
#chat_response = client.chat(
 #   model=model,
 #   messages=messages,
#)

# With streaming
for chunk in client.chat_stream(model=model, messages=messages):
    print(chunk.choices[0].delta.content)

In [ ]:
import gradio as gr
import random
import time
from mistralai.models.chat_completion import ChatMessage
with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.Button("Clear")

    def user(user_message, history):
        return "", history + [[user_message, None]]

    def bot(history):
        messages = [
            ChatMessage(role="user", content=history[-1][0])
        ]
        history[-1][1] = ""
        for chunk in client.chat_stream(model=model, messages=messages):
            if chunk.choices[0].delta.content:
                history[-1][1] += chunk.choices[0].delta.content
            yield history
    msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(
        bot, chatbot, chatbot
    )
    clear.click(lambda: None, None, chatbot, queue=False)
    
demo.queue()
demo.launch()